In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
project_path='/content/drive/My Drive/END/S9/CommonsqnseQA/'
import json
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk
import pickle
import collections
from pandas.io.json import json_normalize
import re
import os
import random
## Lets start the encoder processing for pytorch.
import torch,time, numpy
from torchtext.data import Field,BucketIterator
from torchtext import data
# Manual Seed
SEED = 43
torch.manual_seed(SEED)
import torch.nn as nn

In [ ]:
os.chdir(project_path)

In [ ]:
data_ls=[]
with open('train_rand_split.jsonl', 'r') as f:
    for line in f:
        data_ls.append(json.loads(line))
    extracted_data = json_normalize(data_ls)
    extracted_data.columns = extracted_data.columns.map(lambda x: x.split(".")[-1])
extracted_data.sample(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,answerKey,id,question_concept,choices,stem
4903,D,8758b19506d321d2fc522527aa27d033,jellyfish,"[{'label': 'A', 'text': 'book'}, {'label': 'B'...",Where could a jellyfish stay alive?
7187,E,cb8dbffde4c75a14c33e17be3c48f28b,fiddle,"[{'label': 'A', 'text': 'dance hall'}, {'label...",Where would you go if you need a fiddle?
2001,A,a4df50f416a921ae3d7b3f1603514c66,launch platform,"[{'label': 'A', 'text': 'nasa'}, {'label': 'B'...",The shuttle was inverted onto the launch platf...


In [ ]:
def get_val(chlab,i):
  dict_new={0:'choice_0',1:'choice_1',2:'choice_2',3:'choice_3',4:'choice_4',5:'choice_5'}
  lab=dict_new.get(chlab)
  val=extracted_data[lab][i]
  return val

In [ ]:
num_choices = len(extracted_data['choices'][0])

def get_choices(options, val):
    option = options[int(val)]
    return option.get('text')

choices = np.arange(num_choices)
choices = choices.astype('str')
for c in choices:
    extracted_data['choice_' + c] = extracted_data['choices'].apply(lambda x: get_choices(x, c))
answer_match = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4}

def get_label(answer):
    return answer_match.get(answer)
extracted_data['label'] = extracted_data['answerKey'].apply(lambda x: get_label(x))

extracted_data.sample(3)

,answerKey,id,question_concept,choices,stem,choice_0,choice_1,choice_2,choice_3,choice_4,label
320,B,559ae2b0bfc4b4e66e1acea8b11b485d,reproducing,"[{'label': 'A', 'text': 'overpopulation'}, {'l...","There's a lot to reproducing, but it begins an...",overpopulation,birth,presents,propagation,fun,1
6775,D,730da7dc9ea5754702c0598b64651de3,drop of blood,"[{'label': 'A', 'text': 'during a fight'}, {'l...",Where will more than a drop of blood be spilt?,during a fight,needle,person,battlefield,test tube,3
2055,E,a89fecc27192efbd67629bec05d64b10,fallen leaves,"[{'label': 'A', 'text': 'forest'}, {'label': '...",James raked fallen leaves into a pile. Where m...,forest,roof,garden,tree,ground,4


In [ ]:
lst=[]
shp=extracted_data['label'].shape[0]
for i in range(shp):
  X=extracted_data['label'][i]
  res=get_val(X,i)
  lst.append(res)

extracted_data['lab_value']=lst  


In [ ]:
extracted_data.sample(4)

,answerKey,id,question_concept,choices,stem,choice_0,choice_1,choice_2,choice_3,choice_4,label,lab_value
1529,A,732b435c999691b1184315dca0bf3d0c,police,"[{'label': 'A', 'text': 'board vessels'}, {'la...",The Coast Guard is a police force that often d...,board vessels,direct traffic,ocean,force law,arrest,0,board vessels
3433,E,d328ffdc5d8a18b45200e6701576d2b0,fire,"[{'label': 'A', 'text': 'emits light'}, {'labe...",Bob lights a fire in the wood stove in the win...,emits light,warm hands,boil water,emit light,warm house,4,warm house
4490,C,eff865a55bff055dc740f6217fbc512b,divine,"[{'label': 'A', 'text': 'godless'}, {'label': ...",Someone who doesn't believe in the divine coul...,godless,humdrum,godless,horrible,undivine,2,godless
6412,C,040f2c6071932ad5aabab709d6220b3b,machines,"[{'label': 'A', 'text': 'turn jars'}, {'label'...",Why do manufacturers have many machines?,turn jars,perform work,produce products,answering questions,serve humanity,2,produce products


In [ ]:
SRC = Field(tokenize = 'spacy', 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = 'spacy', 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
#fields = [('stem', SRC),('lab_value',TRG)]
fields = [('src', SRC),('trg',TRG)]

In [ ]:
example = [data.Example.fromlist([extracted_data.stem[i],extracted_data.lab_value[i]], fields) for i in range(extracted_data.shape[0])] 

In [ ]:
QAdataset = data.Dataset(example, fields)

In [ ]:
print(type(QAdataset)) ## the data set is ready.

torchtext.data.dataset.Dataset


In [ ]:
(train_data, valid_data) = QAdataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [ ]:
vars(train_data.examples[6])

{'src': ['a',
  'percussion',
  'instrument',
  'can',
  'be',
  'obtained',
  'in',
  'what',
  'sort',
  'of',
  'place',
  '?'],
 'trg': ['music', 'store']}

In [ ]:
## building vocabulary.
SRC.build_vocab(train_data)
TRG.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in source vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target  vocabulary: {len(TRG.vocab)}")

Unique tokens in source vocabulary: 7562
Unique tokens in target  vocabulary: 3513


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device is' , device)

device is cuda


In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator = BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE, 
     sort_key = lambda x: len(x.src), ## imp lines
    sort_within_batch=True,   ## imp lines 
    device = device)

In [ ]:
## building Seq2Seq model.

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        # Here you can use nn.GRU(emb_dim, hid_dim) or 
        # nn.RNN(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [ ]:
## decoder.
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [ ]:
## Seq2seq.
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [ ]:
## training the model.
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 11,993,785 trainable parameters


In [ ]:
## define the optimizer, padding and the criterion(loss function)
import torch.optim as optim
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
## define the training loop.
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
## Define the eval loop.
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
## define the function to factor the training time.
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
## training the model.
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 8.161 | Train PPL: 3501.868
	 Val. Loss: 8.163 |  Val. PPL: 3509.131
Epoch: 02 | Time: 0m 2s
	Train Loss: 8.161 | Train PPL: 3502.055
	 Val. Loss: 8.163 |  Val. PPL: 3509.131
Epoch: 03 | Time: 0m 2s
	Train Loss: 8.161 | Train PPL: 3501.544
	 Val. Loss: 8.163 |  Val. PPL: 3509.131
Epoch: 04 | Time: 0m 2s
	Train Loss: 8.161 | Train PPL: 3501.129
	 Val. Loss: 8.163 |  Val. PPL: 3509.131
Epoch: 05 | Time: 0m 2s
	Train Loss: 8.161 | Train PPL: 3501.220
	 Val. Loss: 8.163 |  Val. PPL: 3509.131
Epoch: 06 | Time: 0m 2s
	Train Loss: 8.161 | Train PPL: 3501.891
	 Val. Loss: 8.163 |  Val. PPL: 3509.131
Epoch: 07 | Time: 0m 2s
	Train Loss: 8.161 | Train PPL: 3501.764
	 Val. Loss: 8.163 |  Val. PPL: 3509.131
Epoch: 08 | Time: 0m 2s
	Train Loss: 8.161 | Train PPL: 3501.875
	 Val. Loss: 8.163 |  Val. PPL: 3509.131
Epoch: 09 | Time: 0m 2s
	Train Loss: 8.161 | Train PPL: 3501.036
	 Val. Loss: 8.163 |  Val. PPL: 3509.131
Epoch: 10 | Time: 0m 2s
	Train Loss: 8.161 | T